In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-linear-offense-26"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
print(f"openai_api_key={OPENAI_API_KEY}")


In [ ]:
from langchain_openai import ChatOpenAI
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph

llm = ChatOpenAI(model = "gpt-4o-mini", api_key=OPENAI_API_KEY)

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response0 = llm.invoke(messages)
    return {"answer": response0.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])
